In [1]:
from matplotlib.widgets import Slider, Button, RadioButtons, CheckButtons, TextBox # Matplotlib widgets
import matplotlib.pylab as plt # Plotting interface
import numpy as np
from galpy.potential import MiyamotoNagaiPotential, NFWPotential, RazorThinExponentialDiskPotential, BurkertPotential # GALPY potentials
from galpy.potential import calcRotcurve # composed rotation curve calculation for plotting
from astropy import units # Physical/real units data managing
from astropy import table as Table # For fast and easy reading / writing with tables using numpy library
import emcee
import corner
import time
import pandas as pd
import multiprocessing as mp
from scipy.optimize import fsolve


import ipywidgets as widgets


  Referenced from: /Library/Python/3.7/site-packages/libgalpy.cpython-37m-darwin.so
  Reason: image not found' 


In [2]:
#input_params=Table.Table.read('../input_params.txt', format='ascii.tab') # Initial parameters
#input_params

In [3]:
data_rows = [('BULGE', 110000000.0, 1.0, 0.0, 20, 0.495, 70),
             ('THIN DISK', 3900000000.0, 1.0, 5.3, 90, 0.25, 1),
             ('THICK DISK', 39000000000.0, 0.5, 2.6, 20, 0.8, 1),
             ('EXP DISK', 500.0, 0.5, 5.3, 90, 0.0, 0),
             ('DARK HALO', 140000000000.0, 1.0, 13.0, 90, 0.0, 0),
             ('BURKERT HALO', 8000000.0, 1.0, 20.0, 90, 0.0, 0)]
input_params = Table.Table(rows=data_rows, names=('component', 'mass', 'threshold_mass', 'a (kpc)', 'threshold_a', 'b (kpc)', 'threshold_b'))
input_params


component,mass,threshold_mass,a (kpc),threshold_a,b (kpc),threshold_b
str12,float64,float64,float64,int64,float64,int64
BULGE,110000000.0,1.0,0.0,20,0.495,70
THIN DISK,3900000000.0,1.0,5.3,90,0.25,1
THICK DISK,39000000000.0,0.5,2.6,20,0.8,1
EXP DISK,500.0,0.5,5.3,90,0.0,0
DARK HALO,140000000000.0,1.0,13.0,90,0.0,0
BURKERT HALO,8000000.0,1.0,20.0,90,0.0,0


In [4]:
c, mass, delta_mass, radial, delta_radial, vertical, delta_vertical = input_params[0]

In [6]:
import enum

In [7]:
"""class Components_names(enum.Enum):
    Bulge = 'Bulge'
    Thin_disk = 'Thin disk'
    Thick_disk = 'Thick disk'
    Exponential_disk = 'Exponential disk'
    Dark_halo = 'Dark halo'
    Burkert_halo = 'Burkert halo'"""
    

def input_component(component, guess_mass, guess_a, guess_b):
    
    component_mass, component_scale_a, component_scale_b = guess_mass, guess_a, guess_b
    
    print('Set the guess parameters for', component)
    try:
        component_mass = float(input('Mass (in M_sun):'))
    except:
        print('No valid Mass for', component, '. It will be taken the default mass:', component_mass, 'M_sun')
    
    
    try:
        component_scale_a = float(input('Radial Scale Length (in kpc):'))
    except:
        print('No valid Radial Scale Length for', component, '. It will be taken the default Radial Scale Lenght:', component_scale_a, 'kpc')

    if component not in [Components_names.Exponential_disk, Components_names.Dark_halo, Components_names.Burkert_halo]:
        try:
            component_scale_b = float(input('Vertical Scale Length (in kpc):'))
        except:
            print('No valid Vertical Scale Length for', component, '. It will be taken the default Vertical Scale Lenght:', component_scale_b, 'kpc')
    
    return component_mass, component_scale_a, component_scale_b
    
    

In [8]:
c, mass, delta_mass, radial, delta_radial, vertical, delta_vertical = input_params[0]
mass, radial, vertical = input_component(c, mass, radial, vertical)
print(mass, radial, vertical)

c, mass, delta_mass, radial, delta_radial, vertical, delta_vertical = input_params[4]
mass, radial, vertical = input_component(c, mass, radial, vertical)
print(mass, radial, vertical)


Set the guess parameters for BULGE
No valid Mass for BULGE . It will be taken the default mass: 110000000.0 M_sun
No valid Radial Scale Length for BULGE . It will be taken the default Radial Scale Lenght: 0.0 kpc


NameError: name 'Components_names' is not defined